<a href="https://colab.research.google.com/github/SaraBCoutinho/Pesquisa_Mestrado/blob/main/monol%C3%ADticos_predi%C3%A7%C3%B5es_k_folds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [2]:
%%capture
!pip install datasets
from datasets import load_dataset
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from numpy import load
from wordcloud import STOPWORDS, WordCloud
!pip install ftfy
import ftfy
import nltk
import json
import re
%matplotlib inline
import csv
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')
np.random.seed(0)

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV,cross_validate, StratifiedKFold
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import Binarizer, StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import tree
from sklearn import model_selection, naive_bayes, svm
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

from collections import Counter, defaultdict
from os import listdir, makedirs
from os.path import isfile, join, splitext, split
from unicodedata import normalize
import nltk
nltk.download('stopwords')
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from gensim.models import Word2Vec

stopwords_list = stopwords.words('english')
import re
import string
!pip install zeugma
!pip install preprocessing
!pip install function
from zeugma.embeddings import EmbeddingTransformer
from preprocessing import *
from function import *
import itertools

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

### Data

In [9]:
data_kaggle = pd.read_csv('/content/kaggle.csv')
data_kaggle.rename(columns = {'Body':'statement','Label':'label'}, inplace = True)
data_kaggle = data_kaggle.copy().dropna()
data_kaggle = data_kaggle[['statement','label']]
print("frequency of kaggle labels: ", data_kaggle['label'].value_counts())
print("null values kaggle: ", pd.isnull(data_kaggle['statement']).sum(), pd.isnull(data_kaggle['label']).sum(), pd.isnull(data_kaggle).sum())
#data.info()
data_kaggle.reset_index(drop=True, inplace=True)

frequency of kaggle labels:  0    2120
1    1868
Name: label, dtype: int64
null values kaggle:  0 0 statement    0
label        0
dtype: int64


### Pré-processamento

In [10]:
%%time
stopwords_list = stopwords.words('english')
def clean_text(text):
    translate_table = dict((ord(char), None) for char in string.punctuation)
    text = text.translate(translate_table)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    re_url = re.compile(r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*",re.MULTILINE | re.UNICODE)
    re_ip = re.compile("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}")
    text = re_url.sub("URL", text)
    text = re_ip.sub("IPADDRESS", text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = text.lower().split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
#    text = " ".join(stemmed_words)
#    text = text.lower().split()
    stops = set(stopwords.words("english"))
    text = [w for w in stemmed_words if not w in stops and len(w) >= 3]
    text = " ".join(text)
    return text

def tokenizer(text, flg_stemm=True, flg_lemm=False, lst_stopwords=stopwords_list):
    lst_text = text.lower().split()

    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]

    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]

    text = " ".join(lst_text)
    return text

CPU times: user 450 µs, sys: 0 ns, total: 450 µs
Wall time: 460 µs


In [11]:
data_kaggle['statement']=data_kaggle['statement'].map(clean_text)
data_kaggle_to_count=data_kaggle.copy()

def CountOnly(sentence):
  sentence=sentence.split()
  result = Counter(sentence)
  return result

data_kaggle_to_count['statement']=data_kaggle_to_count['statement'].apply(CountOnly)
new_data_kaggle=Counter()

for sentence in data_kaggle_to_count['statement']:
  new_data_kaggle = new_data_kaggle + sentence

list_unique_data_kaggle=[]
for k,v in new_data_kaggle.items():
  if v==1:
    list_unique_data_kaggle.append(k)

total_data_kaggle=0
for k,v in new_data_kaggle.items():
  total_data_kaggle = total_data_kaggle + v

print(total_data_kaggle)
print(len(list_unique_data_kaggle))
print("0 conjunto de dados atual final correspondeu a aproximadamente: ",
      round(((total_data_kaggle-len(list_unique_data_kaggle))/total_data_kaggle)*100,1), "% do conjunto original com um total de: ",(total_data_kaggle-len(list_unique_data_kaggle)),"palavras")

1102335
12994
0 conjunto de dados atual final correspondeu a aproximadamente:  98.8 % do conjunto original com um total de:  1089341 palavras


In [12]:
%%time
def cleanningOnceDataKaggle(statement):
  statement=statement.split()
  result = [word for word in statement if word not in list_unique_data_kaggle]
  result = " ".join(result)
  return result

data_kaggle['statement']=data_kaggle['statement'].apply(cleanningOnceDataKaggle)

CPU times: user 6min 19s, sys: 1.04 s, total: 6min 20s
Wall time: 6min 26s


In [13]:
%%time
#0 fake 1 true
print(data_kaggle['label'].count())
print(data_kaggle['label'].value_counts())
print(data_kaggle['label'].value_counts(1))
data_kaggle=data_kaggle.copy()

3988
0    2120
1    1868
Name: label, dtype: int64
0    0.531595
1    0.468405
Name: label, dtype: float64
CPU times: user 6.29 ms, sys: 0 ns, total: 6.29 ms
Wall time: 6.31 ms


In [14]:
#!rm -rf '/content/Data/'
!mkdir Data
!mkdir Data/kaggle
!mkdir Data/kaggle/F1
!mkdir Data/kaggle/F2
!mkdir Data/kaggle/F3
!mkdir Data/kaggle/F4
!mkdir Data/kaggle/F5
!mkdir Data/kaggle/F6
!mkdir Data/kaggle/F7
!mkdir Data/kaggle/F8
!mkdir Data/kaggle/F9
!mkdir Data/kaggle/F10

In [15]:
kfold = StratifiedKFold(n_splits=10)

for db in ['kaggle']:
    if db == 'kaggle':
      X, y = data_kaggle['statement'], data_kaggle['label']
      folds = kfold.split(X, y)
      dataset_name = 'kaggle'
      path = '/content/Data/'+dataset_name+'/'
      for fold, data in enumerate(folds):
          X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
          X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)

          folder = path + 'F' + str(fold+1) + '/'
          X_train.to_csv(folder+'train.csv')
          X_test.to_csv(folder+'test.csv')
          X_val.to_csv(folder+'val.csv')

          y_train.to_csv(folder+'train_labels.csv')
          y_test.to_csv(folder+'test_labels.csv')
          y_val.to_csv(folder+'val_labels.csv')

          train = pd.concat([X_train,y_train], axis= 1)
          train.to_csv(folder+'train_full.csv')
          val = pd.concat([X_val,y_val], axis= 1)
          val.to_csv(folder+'val_full.csv')
          test = pd.concat([X_test,y_test], axis= 1)
          test.to_csv(folder+'test_full.csv')

### Embeddings

In [16]:
w2v = EmbeddingTransformer('word2vec')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [17]:
glove = EmbeddingTransformer('glove')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [18]:
fasttext = EmbeddingTransformer('fasttext')

[==================================================] 100.0% 958.5/958.4MB downloaded


In [19]:
cv = CountVectorizer(analyzer='word', lowercase=True, stop_words='english')
tfidf =  TfidfVectorizer(analyzer='word', lowercase=True, use_idf=True, stop_words='english')

## PARTE II

### Classificadores

#### Funções

In [24]:
#--Classifiers
#--SVM
def get_svm(cv, tfidf, w2v, glove, fasttext):
    return{
        'CV': {'CLF': SVC(random_state=42, kernel='linear', gamma=0.1, probability=True),'EXT': cv,},
        'TFIDF': {'CLF': SVC(random_state=42, kernel='linear',
                             gamma=0.1, probability=True),'EXT': tfidf,},
        'W2V': {'CLF': SVC(random_state=42, kernel='rbf', gamma=1, probability=True),'EXT': w2v,},
        'GLOVE': {'CLF': SVC(random_state=42, kernel='rbf', gamma=0.5, probability=True),'EXT':  glove,},
        'FAST': {'CLF': SVC(random_state=42, kernel='rbf', gamma=1, probability=True),'EXT': fasttext,}
    }
#--LR
def get_lr(cv,tfidf,glove,w2v,fasttext):
    return {
        'CV': {'CLF': LogisticRegression(random_state=42,multi_class='auto', solver='liblinear', penalty='l1'),'EXT': cv,},
        'TFIDF': {'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),'EXT': tfidf,},
        'GLOVE': {'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),'EXT':  glove,},
        'W2V': {'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),'EXT': w2v,},
        'FAST': {'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),'EXT': fasttext,}
        }
#--RF
def get_rf(cv, tfidf, w2v, glove, fasttext):
    return {'CV': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=20, n_jobs=-1),'EXT': cv,},
        'TFIDF': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50, n_jobs=-1),'EXT': tfidf,},
        'W2V': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50, n_jobs=-1),'EXT': w2v,},
        'GLOVE': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50, n_jobs=-1),'EXT':  glove,},
        'FAST': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50, n_jobs=-1),'EXT': fasttext,}
            }
#--NB
def get_nb(cv, tfidf, w2v, glove, fasttext):
    return {
        'CV': {'CLF': MultinomialNB(alpha=1, fit_prior=False),'EXT': cv,},
        'TFIDF': {'CLF': MultinomialNB(alpha=0.5, fit_prior=False),'EXT': tfidf,},
        'W2V': {'CLF': BernoulliNB(alpha=0.5, fit_prior=True),'EXT': w2v,},
        'GLOVE': {'CLF': BernoulliNB(alpha=0.1, fit_prior=True),'EXT':  glove,},
        'FAST': {'CLF': BernoulliNB(alpha=1, fit_prior=True),'EXT': fasttext,}
    }
#--MLP
def get_mlp(cv, tfidf, w2v, glove, fasttext):
    return {'CV': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40,activation='relu', solver='lbfgs'),'EXT': cv,},
        'TFIDF': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40,activation='logistic', solver='adam'),'EXT': tfidf,},
        'W2V': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40, activation='relu', solver='adam'),'EXT': w2v,},
        'GLOVE': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40, activation='relu', solver='adam'),'EXT':  glove,},
        'FAST': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40, activation='relu', solver='adam'),'EXT': fasttext,}
    }
#--EXTRA
def get_extra(cv, tfidf, w2v, glove, fasttext):
    return {
        'CV': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT': cv,},
        'TFIDF': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT': tfidf,},
        'W2V': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT': w2v,},
        'GLOVE': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT':  glove,},
        'FAST': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT': fasttext,}
    }
#--KNN
def get_knn(cv, tfidf, w2v, glove, fasttext):
    return {
        'CV': {'CLF': KNeighborsClassifier(n_neighbors=3, n_jobs=-1),'EXT': cv,},
        'TFIDF': {'CLF': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),'EXT': tfidf,},
        'W2V': {'CLF': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),'EXT': w2v,},
        'GLOVE': {'CLF': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),'EXT':  glove,},
        'FAST': {'CLF': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),'EXT': fasttext,}
    }

def get_CNN_2(ext, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, MAX_SEQUENCE_LENGTH=300, activation='sigmoid', word_embedding=False, dense=2):
    if word_embedding == False:
        X_ext = ext.get_feature_names()
        model = Word2Vec([X_ext], min_count=1, workers=1, size=300)
    else:
      model = ext.model

    tokenizer = tokenizer
    word_index = tokenizer.word_index

    nb_words = min(MAX_NB_WORDS, len(word_index)) + 1

    embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
    for word, i in word_index.items():
        if word in model.wv.vocab:
            embedding_matrix[i] = model.wv.word_vec(word)


    embedding_layer = Embedding(nb_words,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False
                                )
    cnn = Sequential()
    cnn.add(embedding_layer)
    cnn.add(Dropout(0.2))
    cnn.add(Conv1D(64, 2, padding='valid', activation='relu', strides=1))
    cnn.add(GlobalMaxPooling1D())
    cnn.add(Dense(256))
    cnn.add(Dropout(0.2))
    cnn.add(Activation('relu'))
    cnn.add(Dense(dense))
    cnn.add(Activation(activation))
#    cnn.add(Reshape(None,2))
    cnn.summary()
    cnn.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return cnn

def get_classifier(clf, X, y, ext):
    pipe_clf = Pipeline([('extractor', ext),('clf', clf)])
    pipe_clf.fit(X, y)
    return pipe_clf

def update_pred_proba_2(clfs, label, train, val, test):
    pred_train = pd.DataFrame()
    pred_val = pd.DataFrame()
    pred_test = pd.DataFrame()

    prob_train = pd.DataFrame()
    prob_val = pd.DataFrame()
    prob_test = pd.DataFrame()

    for ext, clf in clfs.items():
        # Predict
        df_pred_train_ = pd.DataFrame(clf.predict(train), columns=["{}-{}".format(label, ext)])
        df_pred_val_ = pd.DataFrame(clf.predict(val), columns=["{}-{}".format(label, ext)])
        df_pred_test_ = pd.DataFrame(clf.predict(test), columns=["{}-{}".format(label, ext)])

        # # Probabilidades
        cols = [
            "{}-{}-{}".format(label, ext, clf.classes_[0]),
            "{}-{}-{}".format(label, ext, clf.classes_[1])
        ]
        df_prob_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
        df_prob_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
        df_prob_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)

        pred_train = pd.concat([pred_train, df_pred_train_], axis=1, sort=False)
        pred_val = pd.concat([pred_val, df_pred_val_], axis=1, sort=False)
        pred_test = pd.concat([pred_test, df_pred_test_], axis=1, sort=False)

        prob_train = pd.concat([prob_train, df_prob_train_], axis=1, sort=False)
        prob_val = pd.concat([prob_val, df_prob_val_], axis=1, sort=False)
        prob_test = pd.concat([prob_test, df_prob_test_], axis=1, sort=False)

    return  pred_train, pred_val, pred_test, prob_train, prob_val, prob_test

#### Classificação  

In [21]:
tokenizer = Tokenizer(num_words=25000)
tokenizer.fit_on_texts(X_train)
print(len(tokenizer.word_index))

23574


In [25]:
%%time
def main():
  for db in ["kaggle"]:
    if db == 'kaggle':
      classe = 'label'
      for fold in ["F1", "F2", "F3", "F4", "F5","F6", "F7", "F8", "F9", "F10"]:

        df_pred_train = pd.DataFrame()
        df_pred_val = pd.DataFrame()
        df_pred_test = pd.DataFrame()

        df_prob_train = pd.DataFrame()
        df_prob_val = pd.DataFrame()
        df_prob_test = pd.DataFrame()

        X_train = pd.read_csv('/content/Data/' + db + "/" + fold+'/train_full.csv')['statement']
        y_train = pd.read_csv('/content/Data/' + db + "/" + fold+'/train_full.csv')['label']
        train = pd.read_csv('/content/Data/' + db + "/" + fold+'/train_full.csv')

        X_test = pd.read_csv('/content/Data/' + db + "/" + fold+'/test_full.csv')['statement']
        y_test = pd.read_csv('/content/Data/' + db + "/" + fold+'/test_full.csv')['label']
        test = pd.read_csv('/content/Data/' + db + "/" + fold+'/test_full.csv')

        X_val = pd.read_csv('/content/Data/' + db + "/" + fold+'/val_full.csv')['statement']
        y_val = pd.read_csv('/content/Data/' + db + "/" + fold+'/val_full.csv')['label']
        val = pd.read_csv('/content/Data/' + db + "/" + fold+'/val_full.csv')


        df_pred_train = pd.concat([df_pred_train, train['label']], axis=1, sort=False)
        df_pred_val = pd.concat([df_pred_val, val['label']], axis=1, sort=False)
        df_pred_test = pd.concat([df_pred_test, test['label']], axis=1, sort=False)

        df_prob_train = pd.concat([df_prob_train, train['label']], axis=1, sort=False)
        df_prob_val = pd.concat([df_prob_val, val['label']], axis=1, sort=False)
        df_prob_test = pd.concat([df_prob_test, test['label']], axis=1, sort=False)

        print('dados ok')

        # SVM
        classifier = get_svm(cv, tfidf, w2v, glove, fasttext)
        clfs = { }

        for ext, clf in classifier.items():
          clfs[ext] = get_classifier(clf['CLF'], X_train, y_train, clf['EXT'])
        pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_2(clfs, "SVM", X_train, X_val, X_test)
        print('svm parte 1 ok')


        df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
        df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
        df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)

        df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
        df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
        df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

        print('svm parte 2 ok')

        # LR
        classifier = get_lr(cv,tfidf,glove,w2v,fasttext)
        clfs = { }

        for ext, clf in classifier.items():
          clfs[ext] = get_classifier(clf['CLF'], X_train, y_train, clf['EXT'])

        pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_2(clfs, "LR", X_train, X_val, X_test)


        df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
        df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
        df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)

        df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
        df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
        df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

        print('lr ok')

        # RF
        classifier = get_rf(cv, tfidf, w2v, glove, fasttext)
        # clfs = { }
        for ext, clf in classifier.items():
            clfs.update({ext: get_classifier(clf['CLF'], X_train, y_train, clf['EXT'])})

        pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_2(clfs, "RF", X_train, X_val, X_test)


        df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
        df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
        df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)

        df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
        df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
        df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

        print('rf ok')

        # NB
        classifier = get_nb(cv, tfidf, w2v, glove, fasttext)
        # clfs = { }
        for ext, clf in classifier.items():
            clfs.update({ext: get_classifier(clf['CLF'], X_train, y_train, clf['EXT'])})

        pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_2(clfs, "NB", X_train, X_val, X_test)


        df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
        df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
        df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)

        df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
        df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
        df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

        print('nb ok')

        # MLP
        classifier = get_mlp(cv, tfidf, w2v, glove, fasttext)
        # clfs = { }
        for ext, clf in classifier.items():
            clfs.update({ext: get_classifier(clf['CLF'], X_train, y_train, clf['EXT'])})

        pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_2(clfs, "MLP", X_train, X_val, X_test)


        df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
        df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
        df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)

        df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
        df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
        df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

        print('mlp ok')

        # Extra
        classifier = get_extra(cv, tfidf, w2v, glove, fasttext)
        # clfs = { }
        for ext, clf in classifier.items():
            clfs.update({ext: get_classifier(clf['CLF'], X_train, y_train, clf['EXT'])})

        pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_2(clfs, "EXTRA", X_train, X_val, X_test)


        df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
        df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
        df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)

        df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
        df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
        df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

        print('extra ok')

        # KNN
        classifier = get_knn(cv, tfidf, w2v, glove, fasttext)
        # clfs = { }
        for ext, clf in classifier.items():
            clfs.update({ext: get_classifier(clf['CLF'], X_train, y_train, clf['EXT'])})

        pred_train, pred_val, pred_test, prob_train, prob_val, prob_test = update_pred_proba_2(clfs, "KNN", X_train, X_val, X_test)


        df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
        df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
        df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)

        df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
        df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
        df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

        print('knn ok')

        #CNN
        MAX_NB_WORDS = 25000
        MAX_SEQUENCE_LENGTH=300

        train_labels = train['label']
        val_labels = val['label']
        test_labels = test['label']

        y_train = to_categorical(train_labels)
        y_val = to_categorical(val_labels)
        y_test = to_categorical(test_labels)

        tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
        tokenizer.fit_on_texts(X_train) #rever por que
        seq_train = tokenizer.texts_to_sequences(X_train)
        seq_val = tokenizer.texts_to_sequences(X_val)
        seq_test = tokenizer.texts_to_sequences(X_test)

        data_train = pad_sequences(seq_train,padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)
        data_val = pad_sequences(seq_val,padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)
        data_test = pad_sequences(seq_test, padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)

        y_validation=to_categorical(y_val, num_classes=2)
        y_training=to_categorical(y_train, num_classes=2)
        y_testing=to_categorical(y_test, num_classes=2)

        # NET
        cnn_cv = get_CNN_2(cv, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', dense=2)
        cnn_tfidf = get_CNN_2(tfidf, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', dense=2)
        cnn_w2v = get_CNN_2(w2v, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True, dense=2)
        cnn_glove = get_CNN_2(glove, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=25, activation='sigmoid', word_embedding=True, dense=2)
        cnn_fast = get_CNN_2(fasttext, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True, dense=2)

        # TRAIN
        #cnn_cv.fit(data_train, y_training, validation_data=(data_val,y_validation), epochs=20, batch_size=20)
        cnn_cv.fit(data_train, y_train, validation_data=(data_val,y_val), epochs=20, batch_size=20)
        cnn_tfidf.fit(data_train, y_train, validation_data=(data_val,y_val), epochs=20, batch_size=20)
        cnn_w2v.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)
        cnn_glove.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)
        cnn_fast.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)

        cols = ["CNN-CV-0", "CNN-CV-1"]

        df_train_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_train), axis=1), columns=["CNN-CV"])
        df_val_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_val), axis=1), columns=["CNN-CV"])
        df_test_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_test), axis=1), columns=["CNN-CV"])
        df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
        df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
        df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
        # Probabilidades
        df_train_ = pd.DataFrame(cnn_cv.predict(data_train), columns=cols)
        df_val_ = pd.DataFrame(cnn_cv.predict(data_val), columns=cols)
        df_test_ = pd.DataFrame(cnn_cv.predict(data_test), columns=cols)
        df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
        df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
        df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

        cols = ["CNN-TF-0", "CNN-TF-1"]
        df_train_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_train), axis=1), columns=["CNN-TFIDF"])
        df_val_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_val), axis=1), columns=["CNN-TFIDF"])
        df_test_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_test), axis=1), columns=["CNN-TFIDF"])
        df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
        df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
        df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
        # Probabilidades
        df_train_ = pd.DataFrame(cnn_tfidf.predict(data_train), columns=cols)
        df_val_ = pd.DataFrame(cnn_tfidf.predict(data_val), columns=cols)
        df_test_ = pd.DataFrame(cnn_tfidf.predict(data_test), columns=cols)
        df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
        df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
        df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

        cols = ["CNN-W2V-0", "CNN-W2V-1"]
        df_train_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_train), axis=1), columns=["CNN-W2V"])
        df_val_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_val), axis=1), columns=["CNN-W2V"])
        df_test_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_test), axis=1), columns=["CNN-W2V"])
        df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
        df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
        df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
        # Probabilidades
        df_train_ = pd.DataFrame(cnn_w2v.predict(data_train), columns=cols)
        df_val_ = pd.DataFrame(cnn_w2v.predict(data_val), columns=cols)
        df_test_ = pd.DataFrame(cnn_w2v.predict(data_test), columns=cols)
        df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
        df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
        df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

        cols = ["CNN-GLOVE-0", "CNN-GLOVE-1"]
        df_train_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_train), axis=1), columns=["CNN-GLOVE"])
        df_val_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_val), axis=1), columns=["CNN-GLOVE"])
        df_test_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_test), axis=1), columns=["CNN-GLOVE"])
        df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
        df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
        df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
        # Probabilidades
        df_train_ = pd.DataFrame(cnn_glove.predict(data_train), columns=cols)
        df_val_ = pd.DataFrame(cnn_glove.predict(data_val), columns=cols)
        df_test_ = pd.DataFrame(cnn_glove.predict(data_test), columns=cols)
        df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
        df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
        df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

        cols = ["CNN-FAST-0", "CNN-FAST-1"]
        df_train_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_train), axis=1), columns=["CNN-FAST"])
        df_val_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_val), axis=1), columns=["CNN-FAST"])
        df_test_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_test), axis=1), columns=["CNN-FAST"])
        df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
        df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
        df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
        # Probabilidades
        df_train_ = pd.DataFrame(cnn_fast.predict(data_train), columns=cols)
        df_val_ = pd.DataFrame(cnn_fast.predict(data_val), columns=cols)
        df_test_ = pd.DataFrame(cnn_fast.predict(data_test), columns=cols)
        df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
        df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
        df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

        print('cnn ok')

        # Probabilidade
        path = "./Data/{}/{}".format(db,fold)
        df_pred_train.to_csv("{}/pred_train.csv".format(path))
        df_pred_val.to_csv("{}/pred_val.csv".format(path))
        df_pred_test.to_csv("{}/pred_test.csv".format(path))

        df_prob_train.to_csv("{}/prob_train.csv".format(path))
        df_prob_val.to_csv("{}/prob_val.csv".format(path))
        df_prob_test.to_csv("{}/prob_test.csv".format(path))

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 11.7 µs


In [ ]:
%%time
if __name__ == "__main__":
    main()